In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
import warnings 
warnings.filterwarnings('ignore')

from functools import reduce
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
data=pd.read_csv('disease_prediction\Training.csv')

In [3]:
emptydictionary={}
for i in range(len(data)):
    y=data.loc[i].reset_index()
    emptydictionary[i]=list(newdata[y[i]==1]['index'])
    
for i in range(len(data)):
    diseas=data['prognosis'][i]
    emptydictionary[i].append(diseas)
    
rawdata=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in emptydictionary.items()]))

rawdata=rawdata.T

NameError: name 'newdata' is not defined

In [ ]:
patientdiseas=[]
for i in range(len(rawdata)):
    temp=list(np.where(rawdata.iloc[i,:].isna())[0])
    if not temp:
        patientdiseas.append((i,17))
    else:
        patientdiseas.append((i,temp[0]-1))
        


In [ ]:
diseases=set()
for z,y in patientdiseas:
    diseases.add(rawdata.iloc[z,y])

In [ ]:
symptomps=[list(rawdata.iloc[i,:].dropna()) for i in range(len(rawdata))]
finalsymptomps=set(reduce(lambda x,y :x+y ,symptomps))

In [ ]:
columnsmissing=set(data.columns.values)-(finalsymptomps-diseases)